### Problem Statement : Increase the sales of the store using a Recommendation System 
#### Include Brand, Gender, Color and Description

In [49]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("default")

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
from urllib.request import urlopen
import requests
from selenium import webdriver
from bs4 import BeautifulSoup

In [3]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [4]:
data = pd.read_csv('Debenham_Womens_Department.csv')

In [5]:
data['PRODUCT_URL'][15]

'https://www.debenhams.com/product/adrianna-papell-structured-cape-sheath_p-bef52ba3-2863-4aca-93d1-203c2a85b854?colour=White'

In [6]:
new_data = data.iloc[:3000,:]

### Web Scraping

In [6]:
def scraper(url):
    # Initialize Chrome webdriver with cookies disabled
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-cookies')
    driver = webdriver.Chrome(options=options)
    
    # Navigate to the desired page
    driver.get(url)
    
    # Get the page source after cookies are disabled
    page_source = driver.page_source
    
    # Create a BeautifulSoup object to parse the HTML
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Close the webdriver
    driver.quit()
    
    desc = soup.find('div',class_='html__HTML-sc-1fx37p7-0 dFHTit')
    text = desc.text
    return text

In [14]:
descriptions = []

In [15]:
for i in tqdm(range(len(new_data))):
    try:
        url = new_data['PRODUCT_URL'][i]
        disc = scraper(url)
        descriptions.append([i,disc])
    except:
        descriptions.append([i,'None'])

100%|██████████| 3000/3000 [15:55:38<00:00, 19.11s/it]  


In [18]:
pd.DataFrame(descriptions).to_csv('Descriptions.csv',index=False)

In [17]:
column_names = ['Index','Description']

In [35]:
scrap_data = pd.read_csv('Descriptions.csv',names=column_names)

In [36]:
scrap_data.drop(scrap_data.index[0],inplace=True)

In [40]:
scrap_data = scrap_data.set_index('Index')

In [41]:
scrap_data

,Description
Index,
0,"In our daydreams, we see this piece on a sandy..."
1,Feel the soft sea breeze when you wear this Ba...
2,Adorned with delicate paisley embroidery and t...
3,When you are looking for something classic wit...
4,"Fierce and fabulous, this tuxedo-style dress i..."
...,...
2995,"You've got the plans, we've got the dresses. F..."
2996,None
2997,"You've got the plans, we've got the dresses. F..."


### Data Preprocessing

In [44]:
prep_data = pd.concat([new_data,scrap_data],axis=1)

In [45]:
prep_data

,BRAND,DEPARTMENT,CATEGORY,SUBCATEGORY,SUBSUBCATEGORY,GENDER,PRODUCT_NAME,PRODUCT_URL,SKU,PRICE_RETAIL,PRICE_CURRENT,DISCOUNT_GIVEN,SORT_BY,BEST_SELLER_CATEGORY_RATING,LISTING_URL,Colour,SKU_VARIANT,SCAN_DATE,TID,Description
0,Accessorize,Womens,Clothing,Clothing,Dresses,Female,Long Sleeve Beach Shirt,https://www.debenhams.com/product/accessorize-...,P-44e9c93a-5a50-4860-babe-9b87477de72c,35.0,35.0,NaN,Best Sellers,1,https://www.debenhams.com/category/womens-dres...,Blue,M5045462665003,2022-11-06 11:18:30,5249214,"In our daydreams, we see this piece on a sandy..."
1,Accessorize,Womens,Clothing,Clothing,Dresses,Female,Schiffly Bardot Dress,https://www.debenhams.com/product/accessorize-...,P-ed1fd2df-a8e3-48ad-aebd-643d5a76d77a,35.0,35.0,NaN,Best Sellers,2,https://www.debenhams.com/category/womens-dres...,Black,M5045462799517,2022-11-06 11:18:30,5249215,Feel the soft sea breeze when you wear this Ba...
2,Accessorize,Womens,Clothing,Clothing,Dresses,Female,Embroidered Tassel Tie Dress,https://www.debenhams.com/product/accessorize-...,P-8806081e-efd7-4b50-86bf-dd6a8904bbb4,54.0,54.0,NaN,Best Sellers,3,https://www.debenhams.com/category/womens-dres...,Blue,M5045462874139,2022-11-06 11:18:30,5249216,Adorned with delicate paisley embroidery and t...
3,Adrianna Papell,Womens,Clothing,Clothing,Dresses,Female,Beaded Chiffon Gown,https://www.debenhams.com/product/adrianna-pap...,P-2a31b6d7-8014-4cc2-8ec5-f3e9fbea09c3,289.0,202.3,Save 30%,Best Sellers,4,https://www.debenhams.com/category/womens-dres...,Red,M0887873725670,2022-11-06 11:18:30,5249217,When you are looking for something classic wit...
4,Adrianna Papell,Womens,Clothing,Clothing,Dresses,Female,Crepe Tuxedo Gown,https://www.debenhams.com/product/adrianna-pap...,P-cf156259-85d7-4887-8e9d-bd5325159adf,199.0,199.0,NaN,Best Sellers,5,https://www.debenhams.com/category/womens-dres...,Navy,M0887873829453,2022-11-06 11:18:30,5249218,"Fierce and fabulous, this tuxedo-style dress i..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Dorothy Perkins,Womens,Clothing,Clothing,Dresses,Female,Cora Long Sleeve Red Floral Midi Dress,https://www.debenhams.com/product/dorothy-perk...,BQQ08406,45.0,36.0,Save 20%,Best Sellers,2996,https://www.debenhams.com/category/womens-dres...,black,BQQ08406-105-22,2022-11-06 11:18:30,5252209,"You've got the plans, we've got the dresses. F..."
2996,Dorothy Perkins,Womens,Clothing,Clothing,Dresses,Female,Petite Dark Rose Textured Shirred Midi Dress,https://www.debenhams.com/product/dorothy-perk...,AQQ46988,45.0,22.5,Save 50%,Best Sellers,2997,https://www.debenhams.com/category/womens-dres...,rose,AQQ46988-158-14,2022-11-06 11:18:30,5252210,None
2997,Dorothy Perkins,Womens,Clothing,Clothing,Dresses,Female,Green Empire Midi Dress,https://www.debenhams.com/product/dorothy-perk...,BQQ07982,45.0,36.0,Save 20%,Best Sellers,2998,https://www.debenhams.com/category/womens-dres...,green,BQQ07982-130-22,2022-11-06 11:18:30,5252211,"You've got the plans, we've got the dresses. F..."
2998,Dorothy Perkins,Womens,Clothing,Clothing,Dresses,Female,Petite Black Satin Puff Sleeve Button Midaxi D...,https://www.debenhams.com/product/dorothy-perk...,AQQ07315,50.0,10.0,Save 80%,Best Sellers,2999,https://www.debenhams.com/category/womens-dres...,black,AQQ07315-105-18,2022-11-06 11:18:30,5252212,Petite Black Satin Puff Sleeve Button Midaxi D...


In [46]:
prep_data.nunique()

BRAND                            27
DEPARTMENT                        1
CATEGORY                          1
SUBCATEGORY                       1
SUBSUBCATEGORY                    1
GENDER                            3
PRODUCT_NAME                   2835
PRODUCT_URL                    3000
SKU                            3000
PRICE_RETAIL                    123
PRICE_CURRENT                   276
DISCOUNT_GIVEN                   57
SORT_BY                           1
BEST_SELLER_CATEGORY_RATING    3000
LISTING_URL                      92
Colour                          196
SKU_VARIANT                    3000
SCAN_DATE                         1
TID                            3000
Description                    2585
dtype: int64

#### Working Data

In [128]:
work_data = prep_data[['BRAND','GENDER','PRODUCT_NAME','PRODUCT_URL','Colour','Description']]

In [129]:
work_data = work_data[work_data['Description']!='None']

In [130]:
work_data.drop(columns='PRODUCT_URL',inplace=True)

In [131]:
work_data

,BRAND,GENDER,PRODUCT_NAME,Colour,Description
0,Accessorize,Female,Long Sleeve Beach Shirt,Blue,"In our daydreams, we see this piece on a sandy..."
1,Accessorize,Female,Schiffly Bardot Dress,Black,Feel the soft sea breeze when you wear this Ba...
2,Accessorize,Female,Embroidered Tassel Tie Dress,Blue,Adorned with delicate paisley embroidery and t...
3,Adrianna Papell,Female,Beaded Chiffon Gown,Red,When you are looking for something classic wit...
4,Adrianna Papell,Female,Crepe Tuxedo Gown,Navy,"Fierce and fabulous, this tuxedo-style dress i..."
...,...,...,...,...,...
2994,Dorothy Perkins,Female,Diamond Printed V Neck Shirred Waist Midi Dress,black,"You've got the plans, we've got the dresses. F..."
2995,Dorothy Perkins,Female,Cora Long Sleeve Red Floral Midi Dress,black,"You've got the plans, we've got the dresses. F..."
2997,Dorothy Perkins,Female,Green Empire Midi Dress,green,"You've got the plans, we've got the dresses. F..."
2998,Dorothy Perkins,Female,Petite Black Satin Puff Sleeve Button Midaxi D...,black,Petite Black Satin Puff Sleeve Button Midaxi D...


In [132]:
work_data['Description'][0]

'In our daydreams, we see this piece on a sandy beach with a cocktail in hand. The perfect airy piece to throw over your swimsuit, simply style it with sandals and some oversized sunnies. Designed in a light linen-blend.'

In [65]:
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()
from nltk.corpus import stopwords

In [79]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [86]:
new_desc = []

In [87]:
for i in tqdm(work_data['Description']):
    new = i.split()
    new = [ps.stem(word) for word in new if word not in set(stopwords.words('english'))]
    new = " ".join(new)
    new_desc.append(new)

100%|██████████| 2853/2853 [02:04<00:00, 23.00it/s]


In [133]:
work_data.replace('[^a-zA-Z]'," ",regex=True,inplace=True)

In [134]:
work_data['BRAND'] = work_data['BRAND'].str.lower()
work_data['GENDER'] = work_data['GENDER'].str.lower()
work_data['Colour'] = work_data['Colour'].str.lower()

In [137]:
work_data['BRAND'] = work_data['BRAND'].apply(lambda x:x.replace(" ",""))

In [139]:
work_data['Meta'] = new_desc

In [141]:
work_data['Meta'] = work_data['BRAND'] + " " + work_data['GENDER'] + " " + work_data['Colour'] + " " + work_data['Meta'] 

In [143]:
work_data.drop(columns=['BRAND','GENDER','Colour','Description'],inplace=True)

In [144]:
work_data

,PRODUCT_NAME,Meta
0,Long Sleeve Beach Shirt,"accessorize female blue in daydreams, see piec..."
1,Schiffly Bardot Dress,accessorize female black feel soft sea breez w...
2,Embroidered Tassel Tie Dress,accessorize female blue adorn delic paisley em...
3,Beaded Chiffon Gown,adriannapapell female red when look someth cla...
4,Crepe Tuxedo Gown,"adriannapapell female navy fierc fabulous, tux..."
...,...,...
2994,Diamond Printed V Neck Shirred Waist Midi Dress,"dorothyperkins female black you'v got plans, w..."
2995,Cora Long Sleeve Red Floral Midi Dress,"dorothyperkins female black you'v got plans, w..."
2997,Green Empire Midi Dress,"dorothyperkins female green you'v got plans, w..."
2998,Petite Black Satin Puff Sleeve Button Midaxi D...,dorothyperkins female black petit black satin ...


#### Text Vectorization

In [153]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000,stop_words='english')

In [161]:
vectors = cv.fit_transform(work_data['Meta']).toarray()

In [160]:
from sklearn.metrics.pairwise import cosine_similarity

In [164]:
cosine_similarity(vectors)

array([[1.        , 0.14638501, 0.18650096, ..., 0.08195375, 0.06201737,
        0.08660254],
       [0.14638501, 1.        , 0.1820063 , ..., 0.18661691, 0.30261377,
        0.22537447],
       [0.18650096, 0.1820063 , 1.        , ..., 0.10189636, 0.11566299,
        0.10767638],
       ...,
       [0.08195375, 0.18661691, 0.10189636, ..., 1.        , 0.16941854,
        0.86746041],
       [0.06201737, 0.30261377, 0.11566299, ..., 0.16941854, 1.        ,
        0.28644595],
       [0.08660254, 0.22537447, 0.10767638, ..., 0.86746041, 0.28644595,
        1.        ]])

In [168]:
similarity = cosine_similarity(vectors)

### Recommendation Model

In [171]:
def recommended_dress(product):
    dress_index = work_data[work_data['PRODUCT_NAME']==product].index[0]
    distance = similarity[dress_index]
    dresses_list = sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in dresses_list:
        print(new_data.iloc[i[0]].PRODUCT_NAME,new_data.iloc[i[0]].PRODUCT_URL)

In [172]:
recommended_dress('Long Sleeve Beach Shirt')

Chunky Rib Knitted Jumper Dress https://www.debenhams.com/product/boohoo-chunky-rib-knitted-jumper-dress_gzz02795?colour=black
Satin Plunge Twist Maxi Dress https://www.debenhams.com/product/boohoo-satin-plunge-twist-maxi-dress_gzz18787?colour=berry
Plus V Bar Square Neck Split Maxi Dress https://www.debenhams.com/product/boohoo-plus-v-bar-square-neck-split-maxi-dress_gzz29347?colour=red
Plus V Neck Smock Dress https://www.debenhams.com/product/boohoo-plus-v-neck-smock-dress_gzz10674?colour=green
Plus Jersey Wrap Skater Dress https://www.debenhams.com/product/boohoo-plus-jersey-wrap-skater-dress_pzz04648?colour=grey marl
